In [2]:
import sys
sys.path.append('/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero')

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import argparse
from transformers import Qwen2VLForConditionalGeneration, Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import json
import pdb

import cv2
from PIL import Image as PILImage
import re
from sam2.sam2_image_predictor import SAM2ImagePredictor
import numpy as np
import matplotlib.pyplot as plt

/home/ksmehrab/miniconda/envs/segzero/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# def parse_args(args=None):
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--reasoning_model_path", type=str, default="/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero/pretrained_models/VisionReasoner-7B")
#     parser.add_argument("--segmentation_model_path", type=str, default="facebook/sam2-hiera-large")
#     parser.add_argument("--text", type=str, default="What can I have if I'm thirsty?")
#     parser.add_argument("--image_path", type=str, default="/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero/assets/food.webp")
#     parser.add_argument("--output_path", type=str, default="./inference_scripts/test_output_multiobject.png")
#     return parser.parse_args(args)

# args = parse_args([])

In [ ]:
# configuration object, to replace args
class Config:
    def __init__(self):
        self.reasoning_model_path = "/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero/pretrained_models/VisionReasoner-7B"
        self.segmentation_model_path = "facebook/sam2-hiera-large"
        self.image_path = "/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/GitRepoLatest/Seg-Zero/assets/food.webp"
        self.output_path = "./test_output.png"

args = Config()

In [ ]:
# This method is to parse the output from default segzero  
def extract_bbox_points_think(output_text, x_factor, y_factor):
    json_match = re.search(r'<answer>\s*(.*?)\s*</answer>', output_text, re.DOTALL)
    if json_match:
        data = json.loads(json_match.group(1))
        pred_bboxes = [[
            int(item['bbox_2d'][0] * x_factor + 0.5),
            int(item['bbox_2d'][1] * y_factor + 0.5),
            int(item['bbox_2d'][2] * x_factor + 0.5),
            int(item['bbox_2d'][3] * y_factor + 0.5)
        ] for item in data]
        pred_points = [[
            int(item['point_2d'][0] * x_factor + 0.5),
            int(item['point_2d'][1] * y_factor + 0.5)
        ] for item in data]
    
    think_pattern = r'<think>([^<]+)</think>'
    think_match = re.search(think_pattern, output_text)
    think_text = ""
    if think_match:
        think_text = think_match.group(1)
    
    return pred_bboxes, pred_points, think_text

In [ ]:
# New output parse for vrpart format 
import json 
import re 
def extract_information_vrpart(output_text, x_factor, y_factor):
    # Extract think tag
    think_pattern = r'<think>([^<]+)</think>'
    think_match = re.search(think_pattern, output_text)
    think_text = think_match.group(1).strip() if think_match else ""
    
    # Extract decide tag
    decide_pattern = r'<decide>([^<]+)</decide>'
    decide_match = re.search(decide_pattern, output_text)
    decide_text = decide_match.group(1).strip() if decide_match else ""
    
    # Extract first_answer tag
    first_answer_pattern = r'<first_answer>\s*(.*?)\s*</first_answer>'
    first_answer_match = re.search(first_answer_pattern, output_text, re.DOTALL)
    first_answer_text = first_answer_match.group(1).strip() if first_answer_match else ""
    
    # Extract criticism tag
    criticism_pattern = r'<criticism>([^<]+)</criticism>'
    criticism_match = re.search(criticism_pattern, output_text)
    criticism_text = criticism_match.group(1).strip() if criticism_match else ""
    
    # Extract final_answer and parse bbox/points
    final_answer_pattern = r'<final_answer>\s*(.*?)\s*</final_answer>'
    final_answer_match = re.search(final_answer_pattern, output_text, re.DOTALL) 
    final_answer_text = final_answer_match.group(1).strip() if final_answer_match else ""

    output_text_parsed = {
        "think": think_text,
        "decide": decide_text,
        "first_answer": first_answer_text,
        "criticism": criticism_text,
        "final_answer": final_answer_text
    }
    
    pred_bboxes = []
    pred_points = []
    
    if final_answer_match:
        data = json.loads(final_answer_match.group(1))
        pred_bboxes = [[
            int(item['bbox_2d'][0] * x_factor + 0.5),
            int(item['bbox_2d'][1] * y_factor + 0.5),
            int(item['bbox_2d'][2] * x_factor + 0.5),
            int(item['bbox_2d'][3] * y_factor + 0.5)
        ] for item in data]
        pred_points = [[
            int(item['point_2d'][0] * x_factor + 0.5),
            int(item['point_2d'][1] * y_factor + 0.5)
        ] for item in data]

    return pred_bboxes, pred_points, output_text_parsed

# test the vrpart output parser
example_output = """<think> The image depicts a suitcase with a telescopic handle extending from its top. The task is to identify and mark the telescopic handle. The handle appears to be a rigid, straight rod that extends upwards from the top of the suitcase.</think> 

<decide>I am finding an "object part"</decide> 

<first_answer>[{"bbox_2d":[406,9,658,196], "point_2d":[517,88]}]</first_answer> 

<criticism>The initial bounding box around the telescopic handle might be too large because it includes parts of the suitcase structure that are not relevant to the handle itself. A more accurate bounding box should only encompass the visible part of the handle, which is the vertical rod.</criticism> 

<final_answer>[{"bbox_2d":[407,10,658,197], "point_2d":[520,87]}]</final_answer>
"""

extract_information_vrpart(example_output, 1.0, 1.0) 

In [5]:
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 0
    return intersection, union

In [6]:
#We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
reasoning_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    args.reasoning_model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.65it/s]


In [ ]:
reasoning_model.eval()

In [8]:
segmentation_model = SAM2ImagePredictor.from_pretrained(args.segmentation_model_path)

In [9]:
# default processer
processor = AutoProcessor.from_pretrained(args.reasoning_model_path, padding_side="left")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
QUESTION_TEMPLATE = \
    "Please find \"{Question}\" with bboxs and points." \
    "Compare the difference between object(s) and find the most closely matched object(s)." \
    "Output the thinking process in <think> </think> and final answer in <answer> </answer> tags." \
    "Output the bbox(es) and point(s) inside the interested object(s) in JSON format." \
    "i.e., <think> thinking process here </think>" \
    "<answer>{Answer}</answer>"

In [ ]:
## Code for getting pascalpart images and object/object parts 
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Models/GLAMM')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Datasets')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines')
    
from pascalpart import get_pascalpart_classes, get_pascalpart_masks
from utils import read_txt_file, save_to_json
from tqdm import tqdm
import os

pascal_image_dir = "/data/Pascal_VOC_2012/VOCdevkit/VOC2012/JPEGImages" # 17125 images
annotations_path= "/data/PartSegmentationDatasets/PascalPart/Annotations_Part"
val_filepath = "/data/PartSegmentationDatasets/PascalPart/val.txt" # 925 images. File contains just the file prefix. Add .jpg extension for images, and .mat extension for annotations
val_filenames = read_txt_file(val_filepath)

## Change 
save_dir = "/data/VLMGroundingProject/BaselineResults/PascalPart/SegZero"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

already_done = [os.path.splitext(f)[0] for f in os.listdir(save_dir)]

req_filenames = [f for f in val_filenames if f not in already_done]

print(f'Already done on {len(already_done)} images. Running on remaining {len(req_filenames)}...')

def combine_masks(masks):
    combined_mask = masks[0]
    for i in range(1, len(masks)):
        combined_mask = combined_mask | masks[i]
    return combined_mask

all_object_outputs = []
all_parts_outputs = []
for filename in tqdm(req_filenames[:10]):
    img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')

    image = PILImage.open(img_filepath)
    image = image.convert("RGB")
    original_width, original_height = image.size
    resize_size = 840
    x_factor, y_factor = original_width/resize_size, original_height/resize_size

    annot_filename = filename + '.mat'
    anno_dict = get_pascalpart_masks(annot_filename, annotations_path, images_path=pascal_image_dir)
    
    # classes_to_detect = get_pascalpart_classes(annot_filename, annotations_path) # This is a dictionary in this format {object: [list of parts]}
    # print(classes_to_detect)
    for obj_name, anno in anno_dict.items():
        obj_masks = anno['object_maps']
        obj_mask = combine_masks(obj_masks)

        gt_mask_list = [obj_mask]
        seg_list = [obj_name]

        parts_masks = anno['parts']
        # print(parts_masks.keys())
        for part_name, masks in parts_masks.items():
            part_mask = combine_masks(masks)
            gt_mask_list.append(part_mask)
            part_full_name = obj_name + "'s " + part_name
            seg_list.append(part_full_name)

        messages = []
        for args_text in seg_list:
            message = [{
                "role": "user",
                "content": [
                {
                    "type": "image", 
                    "image": image.resize((resize_size, resize_size), PILImage.BILINEAR)
                },
                {   
                    "type": "text",
                    "text": QUESTION_TEMPLATE.format(
                        Question=args_text.lower().strip("."),
                        Answer="[{\"bbox_2d\": [10,100,200,210], \"point_2d\": [30,110]}, {\"bbox_2d\": [225,296,706,786], \"point_2d\": [302,410]}]"
                    )    
                }
            ]
            }]
            messages.append(message)
        
        # Preparation for inference
        text = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in messages]
    
        image_inputs, video_inputs = process_vision_info(messages)
 
        inputs = processor(
            text=text,
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        # Inference: Generation of the output
        # add torch inference mode and autocast here for faster inference
        generated_ids = reasoning_model.generate(**inputs, use_cache=True, max_new_tokens=1024, do_sample=False)

        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        batch_output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=False, clean_up_tokenization_spaces=False
        )

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            for id_idx in range(len(batch_output_text)):
                if id_idx == 0:
                    all_outputs = all_object_outputs
                else:
                    all_outputs = all_parts_outputs
                try:
                    bboxes, points, think = extract_bbox_points_think(
                                            batch_output_text[id_idx], 
                                            original_width/resize_size, 
                                            original_height/resize_size
                                        )
                except Exception as e:
                    # add penalty in this situation
                    print("Reasoning error: ", e, "Text: ", batch_output_text[id_idx], "Image: ", filename, "Seg: ", seg_list[id_idx])
                    think = ""
                    intersection = 0
                    union = np.array(gt_mask_list[id_idx]).sum()
                    bbox_iou = 0.0
                    all_outputs.append({
                        "image_id": filename,
                        "ann_id": filename,
                        "seg_id": seg_list[id_idx],
                        "think": think,
                        "intersection": int(intersection),
                        "union": int(union),
                        "bbox_iou": bbox_iou,
                        "visualization_path": None
                    })
                    continue
                try:
                    segmentation_model.set_image(image)
                    mask_all = np.zeros((original_height, original_width), dtype=bool)
                except Exception as e:
                    print("Set image error: ", e, filename)
                    # skip this because the image or mask is not correct
                    continue
                try:
                    for bbox, point in zip(bboxes, points):
                        masks, scores, _ = segmentation_model.predict(
                            point_coords=[point],
                            point_labels=[1],
                            box=bbox
                        )
                        sorted_ind = np.argsort(scores)[::-1]
                        masks = masks[sorted_ind]
                        mask = masks[0].astype(bool)
                        mask_all = np.logical_or(mask_all, mask)
                    gt_mask = np.array(gt_mask_list[id_idx])
                except Exception as e:
                    print("Segmentation error: ", e, filename)
                    # skip this because the image or mask is not correct
                    continue
                try:
                    intersection, union = compute_iou(mask_all, gt_mask)
                except Exception as e:
                    print("Image error: ", e)
                    # skip this because the image or mask is not correct
                    continue 
                
                # save mask_all in save_dir/filename/mask_save_filename 
                mask_save_dir = os.path.join(save_dir, filename)
                os.makedirs(mask_save_dir, exist_ok=True)
                mask_save_filename = seg_list[id_idx].replace("'s", "").replace(" ", "_") + ".npy"
                mask_save_filepath = os.path.join(mask_save_dir, mask_save_filename)
                np.save(mask_save_filepath, mask_all)


                # Create visualization
                import matplotlib.pyplot as plt

                fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                # Original image
                axes[0].imshow(image)
                axes[0].set_title('Original Image')
                axes[0].axis('off')

                # Predicted mask overlay
                axes[1].imshow(image)
                predicted_overlay = np.zeros((mask_all.shape[0], mask_all.shape[1], 4))
                predicted_overlay[mask_all, 0] = 1.0  # Red color for prediction
                predicted_overlay[mask_all, 3] = 0.5  # Alpha transparency
                axes[1].imshow(predicted_overlay)
                axes[1].set_title(f'Predicted: {seg_list[id_idx]}')
                axes[1].axis('off')

                # Ground truth mask overlay
                axes[2].imshow(image)
                gt_overlay = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 4))
                gt_overlay[gt_mask.astype(bool), 1] = 1.0  # Green color for ground truth
                gt_overlay[gt_mask.astype(bool), 3] = 0.5  # Alpha transparency
                axes[2].imshow(gt_overlay)
                axes[2].set_title(f'Ground Truth: {seg_list[id_idx]}')
                axes[2].axis('off')

                plt.tight_layout()
                visualization_path = os.path.join(mask_save_dir, f"{seg_list[id_idx].replace('\'s', '').replace(' ', '_')}_visualization.png")
                plt.savefig(visualization_path)
                # plt.show()
                plt.close()
                
                all_outputs.append({
                    "image_id": filename,
                    "ann_id": filename,
                    "seg_id": seg_list[id_idx],
                    "think": think,
                    "intersection": int(intersection),
                    "union": int(union),
                    "bbox_iou": 0,
                    "visualization_path": visualization_path
                })
        
        # clean GPU memory
        del inputs, generated_ids, generated_ids_trimmed
        torch.cuda.empty_cache()

Already done on 0 images. Running on remaining 925...


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [04:35<00:00, 27.53s/it]


In [12]:
save_to_json(os.path.join("objects_results.json"), all_object_outputs)
save_to_json(os.path.join("parts_results.json"), all_parts_outputs)

In [4]:
def compute_giou(file_paths):
    if isinstance(file_paths, str):
        with open(file_paths, 'r', encoding='utf-8') as f:
            results = json.load(f)
    else:
        # if list, combine all results
        results = []
        for fp in file_paths:
            with open(fp, 'r', encoding='utf-8') as f:
                results.extend(json.load(f))
                
    print(len(results))
    all_ious = []

    # process all items in each file
    for item in results:
        intersection = item['intersection']
        union = item['union']
        
        # calculate IoU of each item
        iou = intersection / union if union > 0 else 0
        all_ious.append({
            # 'image_id': item['image_id'],
            'iou': iou
        })
            
    # calculate gIoU
    gIoU = np.mean([item['iou'] for item in all_ious])
    # calculate cIoU

    # print the results
    print(f"gIoU (average of per image IoU): {gIoU:.4f}")

    return gIoU

In [9]:
compute_giou([f"/data/VLMGroundingProject/BaselineResults/InstructPart/SAM3Text/parts_results_{chunk_id}.json" for chunk_id in range(4)])

600
gIoU (average of per image IoU): 0.7167


np.float64(0.7167356149856086)

In [7]:
# compute_giou([f"/data/VLMGroundingProject/BaselineResults/PascalPart/SAM3/parts_results_{chunk_id}.json" for chunk_id in range(4)])
compute_giou([f"/data/VLMGroundingProject/BaselineResults/PascalPart/SAM3/objects_results_{chunk_id}.json" for chunk_id in range(4)])

1077
gIoU (average of per image IoU): 0.8532


np.float64(0.8532021906003255)

In [17]:
compute_giou("/data/VLMGroundingProject/BaselineResults/InstructPart/VRPart2_OneEpoch/parts_results_3.json")

100
gIoU (average of per image IoU): 0.7686


np.float64(0.768567738378893)

In [6]:
compute_giou("/data/VLMGroundingProject/BaselineResults/PascalPart/VRPart2/parts_results.json")

4110
gIoU (average of per image IoU): 0.0000


np.float64(0.0)

In [ ]:
# pascalpart:
# vr: objects: 86.68, parts: 27.44
# vrip: objects: 85.19, parts: 28.32 
# vrpart: objects: 85.02, parts: 32.69 
# zs qwen2.5 on our prompt: 
# vrpart2_nocontainment (2 epochs): objects: 85.39 , parts: 34.94
# vrpart2 + containment + compactness (2 epochs): objects: 83.29 , parts: 33.69
# sam3: objects: 85.32, parts: 33.05

# instructpart: 
# vr: 59.38 
# vr just trained on ip: 62.33
# vr trained on ip and our reward: 67.14 
# zs qwen2.5 on our prompt:
# vrpart2_nocontainment (2 epochs) (still using the object-aware formulation, just not adding a reward for ensuring part is within object): 73.63
# vrpart2 + containment + compactness (2 epochs): 72.24 (!) # maybe compactness is not helping? part within object should generally help 
# vrpart + containment + compactness (1 epoch): 69.64 
# sam3: 71.67


# partimagenet:
# vr: parts: 45.59
# vr just trained on ip: 
# vr trained on ip and our reward: 50.26
# vrpart2_nocontainment (2 epochs): 53.00
# vrpart2 + containment + compactness (2 epochs):